# String distances




In [8]:
!pip install editdistance

In [9]:
import nltk
import numpy as np
import pandas as pd
import editdistance

In [10]:
words = nltk.corpus.words.words()

In [11]:
pd.DataFrame(words).to_csv("words_nltk.csv")

# BK tree

## How to create a BK-tree

- take any word from your set and use it in as your root node. Add word to the tree based on their distance to the root.


### Example

- Let us consider we started a tree with the word set [**book**, **books**, **cake**] then my tree would look like this if I started by making the word **book**:

```
       1 
book ----- books
    |
    | 4
    ----cake
```

- Add **boo**. Now we want to add the word **boo** which is at distace 1 from book. Notice though that there is already **book** at distance 1. The BK tree has to respect that every node have all children with different distances. If there is a colission (like we have now) then the new word must become a children of the node with which there was the colition. In this case, a children of **book**. The new weight from **books** to **boo** has to be `edit_dis(books,boo)=2`.

```
       1           2
book ----- books ----- boo
    |
    | 4
    ----cake
```

#### Adding [**cape**,**cart**,**boon**,**cook**].

- Add **cape**. Notice `edit_dis(books,cape)=4` but **cake** is altready at distance 4. Therefore **cape** needs to be a child of **cake** with a weighted edge of weight `edit_dis(cake,cape)=1`.
```
       1           2
book ----- books ----- boo
    |
    | 4        1  
    ----cake ----- cape
```

- Add **cart**. Notice `edit_dis(books,cart)=4`. Therefore **cart** needs to be a children of **cake** (or any subchildren of cake). Since `edit_dis(cake,cart)=2` then :
```
       1           2
book ----- books ----- boo
    |
    | 4        1  
    ----cake ----- cape
           |
           | 2        
           ----- cart
```

- Add **boon**. Notice  `edit_dis(book,boon)=1`. Therefore **boon** has to be a descendant of **books** (because books is a son of book with weight 1). Notice `edit_dis(books,boon)=2` but there is already node **boo** at distance 2. Therefore **boon** needs to be a descendant of **boo**. Notice `edit_dis(boo,boon)=1`.

```
       1           2         1
book ----- books ----- boo ----- boon
    |                   
    | 4        1  
    ----cake ----- cape
           |
           | 2        
           ----- cart
```

- Add **cook**.  Notice  `edit_dis(book,cook)=1` but book contains a descendant at distance 1, books, so cook has to be a dascendant of books.  Notice  `edit_dis(books,cook)=2` but books has a descendant at distance 2, **boo**, so cook has to be a descendant of boo.   Notice  `edit_dis(boo,cook)=2` and boo has no other descendant at distance 2. Therefore we can add cook as descendant of boo with weight 2.

```
       1           2         1
book ----- books ----- boo ----- boon
    |                    |
    |                    |  2
    |                    ------ cook
    | 4        1  
    ----cake ----- cape
           |
           | 2        
           ----- cart
```


## How to code a BK tree

Now we would like to know how do we construct a bk-tree in python and store it. A natural way to store a tree is using many tuples concatenated. Since a tree 




##### Exercise: fill in the BKTree class to create a tree

In [12]:
edit_distance = editdistance.eval

In [19]:
edit_distance('abc', 'abb')

1

In [20]:
class BKTree:
    def __init__(self, distfn, words):
        self.distfn = distfn

        it = iter(words)
        root = next(it)
        self.tree = (root, {})

        for i in it:
            self._add_word(self.tree, i)

    def _add_word(self, parent, word):
        pword, children = parent
        d = self.distfn(word, pword)
        
        if d in children:
            self._add_word(children[d], word)
            
        else:
            children[d] = (word, {})        

In [21]:
%%time
t = BKTree(edit_distance, words)

CPU times: user 2.23 s, sys: 65.3 ms, total: 2.3 s
Wall time: 2.3 s


In [22]:
len(t.tree)

2

In [26]:
branches_from_root = t.tree[1]
branch_23          = branches_from_root[23]
branch_24          = branches_from_root[24]

In [27]:
len(branches_from_root)

24

In [28]:
branch_23

('anthropomorphologically',
 {20: ('blepharosphincterectomy', {16: ('epididymodeferentectomy', {})}),
  21: ('formaldehydesulphoxylic',
   {19: ('pericardiomediastinitis', {}),
    22: ('Pseudolamellibranchiata', {2: ('pseudolamellibranchiate', {})}),
    20: ('transubstantiationalist', {})}),
  19: ('gastroenteroanastomosis',
   {15: ('macracanthrorhynchiasis', {}),
    17: ('pancreaticoduodenostomy', {}),
    20: ('phenolsulphonephthalein', {})}),
  18: ('hematospectrophotometer',
   {22: ('scientificogeographical', {}), 19: ('thymolsulphonephthalein', {})}),
  13: ('pathologicohistological', {}),
  14: ('philosophicotheological', {})})

In [29]:
branch_24

('formaldehydesulphoxylate',
 {21: ('pathologicopsychological', {}),
  22: ('scientificophilosophical', {}),
  18: ('tetraiodophenolphthalein', {}),
  20: ('thyroparathyroidectomize', {})})

## Searching in a BK tree

Now in order to search all words that appear at distance less or equal than a tolerance T form a query word `q` we need to visit all nodes n that are at distance D(n,q)+-N.

Let us consider word q=caqe, T=1, candidates = [], search=[book]

Select candidate **book** from search=[book]
- edit_dist(book,caqe) = 4 => candidates is not updated
    - Crawl all children of book at distance I=[4-1,4+1]=[3,5]
    - There is a single node, search=[book,cake], inside I.
    - search = [book,cake]\book = [cake]
    
Select candidate **cake** from search=[cake]
- edit_dist(cake,caqe) = 1 => candidates =[cake]
    - Crawl all children of book at distance I=[1-1,1+1]=[0,2]
    - There are 2 possible nodes, search=[cape, cart]


Select candidate **cape** from search=[cape,cart]
- edit_dist(cape,caqe) = 1 => candidates =[cake, cape]
    - Crawl all children of cape at distance I=[1-1,1+1]=[0,2]
    - Cape has no children
    - search = [cape, cart]\cape = [cart]
    
    
Select candidate **cart** from search=[cart]
- edit_dist(cart,caqe) = 2 => candidates is not updated
    - Crawl all children of cape at distance I=[1-1,1+1]=[0,2]
    - Caqe has no children
    - search = [cart]\cart = []


Select candidate... There is no candidate! stop process. 

The resulting set of possible candidates at distance 1 are: [cape,cake].

Notice that we ended up computing 4 edit distances yet we have 8 nodes.



### Making queries 

In [30]:
visited_nodes = []
class BKTree:
    def __init__(self, distfn, words):
        self.distfn = distfn

        it = iter(words)
        root = next(it)
        self.tree = (root, {})

        for i in it:
            self._add_word(self.tree, i)

    def _add_word(self, parent, word):
        pword, children = parent
        d = self.distfn(word, pword)
        
        if d in children:
            self._add_word(children[d], word)
            
        else:
            children[d] = (word, {})       
        
    def _search_descendants(self, parent, n, distance, query_word):
        
        node_word, children_dict = parent
        dist_to_node = distance(query_word, node_word)
        visited_nodes.append(node_word)
        results = []
        if dist_to_node <= n:
            results.append((dist_to_node, node_word))
        
        # TODO BEGIN ------------------------------------------
        
        for i in range(dist_to_node - n, dist_to_node + n):
            child = children_dict.get(i)
            if child is not None:
                results.extend(self._search_descendants(child, n, distance, query_word))
        
        # TODO END --------------------------------------------

          
        return results
            
    def query(self, query_word, n):
        # sort by distance
        return sorted(self._search_descendants(self.tree, n, self.distfn, query_word))

In [31]:
%%time
t = BKTree(edit_distance,words)

CPU times: user 2.08 s, sys: 54.5 ms, total: 2.14 s
Wall time: 2.17 s


In [32]:
%%time
visited_nodes = []
print(t.query("senzorial", 1))

[(1, 'sensorial')]
CPU times: user 621 µs, sys: 29 µs, total: 650 µs
Wall time: 655 µs


In [33]:
len(visited_nodes), len(words)

(402, 236736)

Compare it with the nltk.edit_distance

In [34]:
%%time
t_nltk = BKTree(nltk.distance.edit_distance, words)

CPU times: user 1min 32s, sys: 318 ms, total: 1min 32s
Wall time: 1min 32s


In [35]:
%%time
t_nltk.query("senzorial", 1)

CPU times: user 17.8 ms, sys: 379 µs, total: 18.2 ms
Wall time: 17.9 ms


[(1, 'sensorial')]

### Comparing exhaustive computation vs bktree with the same edit distance function

In [54]:
from typing import List

def get_candidates_exhaustive(query:str , max_dist: int, vocabulary: List[str], sort_candidates=True):
    """
    Compute the candidate words from the vocabulary at most `max_dist` away from the input token.
    Then it filters candidates by the computed values in the distance function.
    This function is mainly used for benchmarking purposes.
    :return: list of candidate words.
    """
    query = query.lower()
    distance_token_to_words = {
        word: edit_distance(query, word) for word in vocabulary
    }
    min_dist = min(distance_token_to_words.values())
    if min_dist <= max_dist:
        if sort_candidates:
            result = sorted(
                [
                    (distance, word)
                    for word, distance in distance_token_to_words.items()
                    if distance <= max_dist
                ]
            )
        else:
            result = [
                word
                for word, distance in distance_token_to_words.items()
                if distance <= max_dist
            ]
        return result

In [61]:
word = "hamer"
max_dist = 1
sort_candidates=True
        
candidates_ext = get_candidates_exhaustive(word, max_dist, words, sort_candidates)
candidates_ext

[(1, 'haler'),
 (1, 'hame'),
 (1, 'hamel'),
 (1, 'hammer'),
 (1, 'hamper'),
 (1, 'harmer'),
 (1, 'hater'),
 (1, 'haver'),
 (1, 'hawer'),
 (1, 'hazer'),
 (1, 'homer'),
 (1, 'namer'),
 (1, 'shamer'),
 (1, 'tamer')]

Notice that we have an issue with "hamer" since it is (in edit distance) as close as "hammer"  than "haler".

We would like to, somehow, choose wisely among a set of candidates.

In [56]:
candidates_bktree = t.query("hamer", 1)
candidates_bktree

[(1, 'haler'),
 (1, 'hame'),
 (1, 'hamel'),
 (1, 'hammer'),
 (1, 'hammer'),
 (1, 'hamper'),
 (1, 'harmer'),
 (1, 'hater'),
 (1, 'haver'),
 (1, 'hawer'),
 (1, 'hazer'),
 (1, 'homer'),
 (1, 'namer'),
 (1, 'shamer'),
 (1, 'tamer')]

In [60]:
%timeit  get_candidates_exhaustive(word, max_dist, words, sort_candidates)
%timeit  t.query(word, 1)

468 ms ± 37.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
9.78 ms ± 146 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)



### Correcting a pharse

Notice that there are several words at the same distance. Picking the correct word is not the aim for today's class. The aim is to find a small list that contains the correct word.

In [86]:
phrase = "the man wentt to the storr to buy a hamer and some nails"

In [87]:
%%time
correction = []
for w in phrase.split(" "):
    if w in words:
        correction.append(w)
    else:
        w_similar = get_candidates_exhaustive(w, 2, words, True)
        #import pdb;pdb.set_trace()
        if len(w_similar)>0:
            w_corrected = w_similar[0][1]
            correction.append(w_corrected)
        else:
            # no word found, simply append the unedited word
            correction.append(w)

print("Original:   ",phrase)
print("Corrected:  "," ".join(correction))
print("\nTimming")

Original:    the man wentt to the storr to buy a hamer and some nails
Corrected:   the man went to the store to buy a haler and some nail

Timming
CPU times: user 2.1 s, sys: 19.6 ms, total: 2.12 s
Wall time: 2.14 s


In [88]:
%%time
correction = []
for w in phrase.split(" "):
    if w in words:
        correction.append(w)
    else:
        w_similar = t.query(w,2)
        #import pdb;pdb.set_trace()
        if len(w_similar)>0:
            w_corrected = w_similar[0][1]
            correction.append(w_corrected)
        else:
            # no word found, simply append the unedited word
            correction.append(w)


print("Original:   ",phrase)
print("Corrected:  "," ".join(correction))
print("\nTimming")

Original:    the man wentt to the storr to buy a hamer and some nails
Corrected:   the man went to the store to buy a haler and some nail

Timming
CPU times: user 339 ms, sys: 6.06 ms, total: 345 ms
Wall time: 346 ms
